In [1]:
import keras

import pandas as pd
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import csv

2024-04-06 21:57:30.813336: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-06 21:57:30.813511: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-06 21:57:30.816029: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-06 21:57:30.848016: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-06 21:57:31.407605: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

Cargamso el dataset:

In [2]:
iris = datasets.load_iris()
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica

:Summary Statistics:

============== ==== ==== ======= ===== ====================
                Min  Max   Mean    SD   Class Correlation
============== ==== ==== ======= ===== ====================
sepal length:   4.3  7.9   5.84   0.83    0.7826
sepal width:    2.0  4.4   3.05   0.43   -0.4194
petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
============== ==== ==== ======= ===== ====================

:Missing Attribute Values: None
:Class Distribution: 33.3% for each of 3 classes.
:Cr

Creamos el dataset en pandas y lo sufleamos

In [3]:
iris_df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                        columns= iris['feature_names'] + ['target'])
print("Numero de elementos de cada clase")
print(iris_df['target'].value_counts())

Numero de elementos de cada clase
target
0.0    50
1.0    50
2.0    50
Name: count, dtype: int64


In [4]:
def normalize_l2(df, columns):
  """
  Normalizes specified columns in a dataframe using L2 norm.

  Args:
      df: The pandas dataframe.
      columns: List of column names to normalize.

  Returns:
      A new dataframe with normalized columns.
  """
  df_norm = df.copy()
  for col in columns:
    # Calculate L2 norm for each row
    norm = df[col].apply(lambda x: np.sqrt(sum(x**2 for x in df[col])))

    # Avoid division by zero (rows with all zeros)
    norm[norm == 0] = 1

    # Normalize the column values
    df_norm[col] = df[col] / norm
  return df_norm

In [5]:
iris_shuffleados = iris_df.sample(frac=1, random_state=0)

tamañoIris = len(iris_shuffleados)
tamañoResto = int(tamañoIris * 0.8)


iris_trainAll = iris_shuffleados.iloc[:tamañoResto]
iris_pruebaAll = iris_shuffleados.iloc[tamañoResto:]


tamañoEntrenamiento = int(tamañoResto * 0.8)

irisEntrenamientoAll = iris_trainAll.iloc[:tamañoEntrenamiento]
irisValidacionAll = iris_trainAll.iloc[tamañoEntrenamiento:]

print(len(iris_trainAll), len(iris_pruebaAll))
print(len(irisEntrenamientoAll), len(irisValidacionAll))

120 30
96 24


In [6]:
irisEntrenamientoAll = irisEntrenamientoAll.to_numpy()
x = irisEntrenamientoAll[:, :-1]
y = irisEntrenamientoAll[:, -1]

irisValidacionAll = irisValidacionAll.to_numpy()
x_val = irisValidacionAll[:, :-1]
y_val = irisValidacionAll[:, -1]


iris_pruebaAll = iris_pruebaAll.to_numpy()
x_test = iris_pruebaAll[:, :-1]
y_test = iris_pruebaAll[:, -1]

In [7]:
y

array([2., 1., 0., 2., 0., 2., 0., 1., 1., 1., 2., 1., 1., 1., 1., 0., 1.,
       1., 0., 0., 2., 1., 0., 0., 2., 0., 0., 1., 1., 0., 2., 1., 0., 2.,
       2., 1., 0., 1., 1., 1., 2., 0., 2., 0., 0., 1., 2., 2., 2., 2., 1.,
       2., 1., 1., 2., 2., 2., 2., 1., 2., 1., 0., 2., 1., 1., 1., 1., 2.,
       0., 0., 2., 1., 0., 0., 1., 0., 2., 1., 0., 1., 2., 1., 0., 2., 2.,
       2., 2., 0., 0., 2., 2., 0., 2., 0., 2., 2.])

In [19]:

my_callbacks = [
    keras.callbacks.EarlyStopping(patience=2),
    keras.callbacks.ModelCheckpoint(filepath='modelo.{epoch:02d}-{val_loss:.2f}.keras'),
    keras.callbacks.TensorBoard(log_dir='./logs'),
]

neuronas = [12, 30, 48]
learning_rates = [0.01, 0.05, 0.1]
epochs = [100, 200, 300]
momentums = [0.1, 0.25, 0.5]
for neurona in neuronas:
    for i in [10, 15 , 20]:
       for lr in learning_rates:
            for epoch in epochs:
                for momentum in momentums:
                    modelo = keras.Sequential()
                    modelo.add(keras.layers.Dense(units=16, activation="relu", input_shape=(4,)))
                    for j in range(i):
                        modelo.add(keras.layers.Dense(neurona, activation="relu"))
                        if j == i -1:
                            print("-" * 100 + "\n")
                            print("modelo con " + str(neurona) +" neuronas por capa, " + str(i) +" capas")
                            modelo.add(keras.layers.Dense(3, activation="sigmoid"))
                            
                            nombre = 'str(neurona) neuronas  + i +  capas'
                            modelo.name = str(neurona)
                            
                            optimizer = keras.optimizers.SGD(learning_rate=lr, momentum=momentum)
                            modelo.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
                            modelo.fit(x=x, y=y, epochs=epoch, batch_size=32, validation_data=(x_val, y_val))
                            
                            loss, accuracy = modelo.evaluate(x_test, y_test)
                            print("Validation Loss:", loss)
                            print("Validation Accuracy:", accuracy)
                            with open("data.csv", "a", newline="") as csvfile:
                                writer = csv.writer(csvfile)
                                #writer.writerow(["Capas", "Neuronas por capa", "loss", "accuracy"])
                                writer.writerow([i, neurona, lr, epoch, momentum ,loss, accuracy])




SyntaxError: incomplete input (3143108488.py, line 17)

In [9]:



model = keras.Sequential()
model.add(keras.layers.Dense(12, activation='relu', input_dim=4))
model.add(keras.layers.Dense(3, activation='softmax'))
sgd = keras.optimizers.SGD(learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov=True)
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


/home/david/Trabajos/IA2/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py:34: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [10]:
y_train = keras.utils.to_categorical(y, num_classes = 3)

In [11]:
# Define the number of features and classes
num_features = 4
num_classes = 3

# Sample data (replace with your actual data)
x_train = x  # Your training data with shape (n_samples, 4)
y_train = y  # Your integer encoded labels with shape (n_samples,)

# Define the model
model = keras.Sequential([
  keras.layers.Dense(units=16, activation="relu", input_shape=(num_features,)),  # Hidden layer with 16 neurons and ReLU activation
  keras.layers.Dense(units=8, activation="relu"),  # Another hidden layer with 8 neurons and ReLU activation
  keras.layers.Dense(units=num_classes, activation="softmax")  # Output layer with 3 neurons and softmax activation for multi-class classification
])

# Compile the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32)

# Use the model for prediction
predictions = model.predict(x_test)  # Replace x_test with your testing data


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3932 - loss: 1.0721  
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4010 - loss: 1.0415 
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3385 - loss: 1.0843 
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3464 - loss: 1.0677 
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3464 - loss: 1.0575 
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3659 - loss: 1.0432 
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3815 - loss: 1.0325 
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3932 - loss: 1.0202 
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3815 - loss: 1.0239 
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3307 - loss: 1.0329 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


In [12]:
predictions

array([[0.27135062, 0.39578465, 0.3328648 ],
       [0.22168544, 0.35176438, 0.4265502 ],
       [0.2715113 , 0.3798096 , 0.34867916],
       [0.28669265, 0.38091946, 0.33238792],
       [0.2231554 , 0.3476318 , 0.4292128 ],
       [0.28376037, 0.3853013 , 0.3309383 ],
       [0.18861404, 0.3208438 , 0.49054226],
       [0.24526338, 0.35524437, 0.39949226],
       [0.25101888, 0.35579234, 0.39318883],
       [0.2613536 , 0.3589644 , 0.37968194],
       [0.19423376, 0.31968656, 0.4860797 ],
       [0.19588566, 0.3150252 , 0.4890891 ],
       [0.24263038, 0.35146594, 0.4059036 ],
       [0.22558987, 0.34614608, 0.42826402],
       [0.28937986, 0.3846824 , 0.32593775],
       [0.26538694, 0.3603696 , 0.3742435 ],
       [0.19185635, 0.31647262, 0.49167103],
       [0.22006501, 0.33700457, 0.4429305 ],
       [0.28266886, 0.38620105, 0.33113015],
       [0.24597503, 0.3606101 , 0.39341483],
       [0.2424634 , 0.35789794, 0.39963874],
       [0.21714556, 0.34263822, 0.44021618],
       [0.